# [문제] Managing Conversation History
- trim_messages()
- RunnablePassthrough
- itemgetter()

세션id설정하고, 대화 진행 (multi-turn)  
llm모델이 과거 대화를 기억 못 하는 상황 만들기.

In [ ]:
## 1. 모듈 import
from operator import itemgetter
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough
from langchain_core.messages import SystemMessage, HumanMessage, trim_messages


## 2. .env 파일에서 환경변수 읽어오기
load_dotenv()


## 3. 세션별 대화 히스토리를 저장할 임시 메모리 저장소
## type: dict 
store = {}

## 4. 함수 정의: 세션 ID에 따라 대화 히스토리 반환
def get_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

## 5. prompt template 정의
messages = [
    ('system', '''당신은 이력서 작성 컨설턴트입니다.
아래 정보를 바탕으로 지원자 입장에서 2000자 이내로 이력서를 작성합니다.
문장은 자연스럽고 매끄럽게 작성합니다.'''),
	MessagesPlaceholder(variable_name="messages"),
	# ('placeholder', '{chat_history}'),
	('user', '{query}')
]

prompt = ChatPromptTemplate.from_messages(messages=messages)

## 6. ChatOpenAI 인스턴스 생성: 모델 생성
llm = ChatOpenAI(
    model='gpt-4o',
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)
##  7. trim 설정
trimmer = trim_messages(
    max_tokens=40,
    strategy="last",
    token_counter=llm,
    include_system=True,
    allow_partial=False,
    start_on="Human",
)


## 8. chain 구성
chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages") | trimmer)
    | prompt
    | llm
)

## 9. chain에 대화 히스토리 기능을 래핑해서 추가
with_message_history = RunnableWithMessageHistory(
    chain,
    get_history,
    input_messages_key='query',
    history_messages_key='messages'
)


## 10. chain 실행
while True:
    query = input('이력서 작성 컨설턴트입니다. 질문하세요. [종료: S] >>> ')

    if query.upper() == 'S':
        break

    response = with_message_history.invoke(
        {'query': query},
        config={'configurable': {'session_id': '1234'}}      
    )
    print('\n' + '+' * 50 + '\n')

In [40]:
for messages in get_history("1234").messages:
  print(f'[{messages.type.upper()}] : {messages.content}')
  print('\n' + '+' * 50 + '\n')

[HUMAN] : 이력서 작성시 필요한 팁

++++++++++++++++++++++++++++++++++++++++++++++++++

[AI] : 이력서를 작성할 때 아래의 팁들을 참고하시면 도움이 될 것입니다:

1. 맞춤형 이력서 작성: 지원하는 직무에 맞춰 이력서를 조정하세요. 중요 경력이나 기술은 해당 직무와 관련된 것을 강조하는 것이 좋습니다.

2. 간결함 유지: 이력서는 최대 1~2페이지로 유지하세요. 짧고 집중된 내용이 중요합니다.

3. 명확한 연락처 정보: 이름, 전화번호, 이메일 주소 등을 정확히 기재하세요.

4. 프로페셔널한 형식: 깔끔하고 읽기 쉬운 서식을 사용하세요. 표준 글꼴과 일관된 포맷을 유지하는 것이 중요합니다.

5. 경력 기술: 각 직무에 대한 책임과 성과를 명확하게 기술하되, 가능한 경우 구체적인 수치나 결과를 제시하세요.

6. 맞춤형 자기소개: 자신이 왜 이 직무에 적합한 후보자인지를 설명하는 간단한 자기소개를 포함하세요.

7. 중요 기술 및 자격증: 지원하는 직무와 관련된 기술과 자격증을 강조하세요.

8. 철자 및 문법 검사: 이력서 제출 전 철자 오류나 문법적 실수가 없는지 꼼꼼히 검토하세요.

9. 관련 경험 강조: 모든 경력을 나열할 필요는 없으며, 지원 직무와 관련성이 높은 경험에 집중하세요.

10. 최신 정보 반영: 이력서의 정보가 최신인지 확인하세요, 특히 연락처 및 최근 경력 부분을 주의 깊게 살펴보세요.

전문가의 피드백을 받거나 이력서를 여러 번 검토하여 완성도를 높이는 것도 좋은 방법입니다.

++++++++++++++++++++++++++++++++++++++++++++++++++

[HUMAN] : 이력서를 작성할 때 가장 중요한 것

++++++++++++++++++++++++++++++++++++++++++++++++++

[AI] : 이력서를 작성할 때 가장 중요한 요소는 다음과 같습니다:

1. **명확성**: 이력서는 명확하고 이해하기 쉬워야 합니다. 고용주는 많은 이력서를 검

In [41]:
response.content

'[이력서]\n\n김지원  \n서울특별시 강남구 역삼동  \n이메일: kjw@example.com  \n전화: 010-1234-5678  \n\n---\n\n**경력 목표**\n\n빠르게 변화하는 IT 환경에서 창의적이고 효율적인 소프트웨어 솔루션을 개발하여 기업의 성공에 기여하는 것을 목표로 합니다. 특히 인공지능 및 머신러닝 분야에서 혁신적인 프로젝트를 이끌고, 지속적인 학습과 협업을 통해 개인 및 조직의 성장을 도모하고자 합니다.\n\n---\n\n**학력**\n\n서울대학교 컴퓨터공학과, 학사  \n졸업: 2020년 2월  \n\n서울과학기술대학교 대학원, 컴퓨터공학과, 석사  \n졸업: 2023년 2월  \n\n---\n\n**경력**\n\n삼성전자, 소프트웨어 엔지니어  \n서울, 2020년 3월 - 현재  \n\n- 인공지능 기반의 데이터 분석 시스템 개발에 참여하여, 처리 속도를 30% 향상시켰습니다.\n- 팀 내에서 코딩 표준화 프로젝트를 주도해 코드 품질을 개선하고 유지보수 시간을 단축시켰습니다.\n- 머신러닝 알고리즘을 활용한 이미지 인식 프로젝트에서 주요 기여를 통해 프로젝트를 성공적으로 완료했습니다.\n\n---\n\n**기술 스택**\n\n- 프로그래밍 언어: Python, Java, C++\n- 데이터베이스: MySQL, MongoDB\n- 도구 및 기술: TensorFlow, Keras, Git, Docker\n- 인공지능 및 머신러닝 알고리즘에 대한 깊은 이해와 실전 경험\n\n---\n\n**수상 및 자격**\n\n- ACM-ICPC 국제 대학생 프로그래밍 경시대회 본선 진출, 2019\n- 한국정보올림피아드 금상, 2018\n- 정보처리기사 자격증, 2021\n\n---\n\n**프로젝트 경험**\n\n1. **자연어 처리 기반 챗봇 개발**  \n   삼성전자 내부 고객센터에서 활용되는 챗봇을 개발하여 응답률을 40% 향상시켰습니다.  \n   - 자연어 처리 엔진 설계 및 구현\n   - 사용자 피드백을 기반으로 한 지속적